### Perturb_seq Dataset Download and Preprocessing 

Preprocessing data code from repository suinleelab **contrastiveVI**: https://github.com/suinleelab/contrastiveVI/blob/main/contrastive_vi/data/datasets/norman_2019.py

The link to download the data is in the code. The Perturb_seq data is from Gene Expression Omnibus (GEO) public genomics data repository (https://ftp.ncbi.nlm.nih.gov/geo/series/GSE133nnn/GSE133344/suppl). The particular dataset for this experiment are "GSE133344_filtered_matrix.mtx.gz", "GSE133344_filtered_genes.tsv.gz", "GSE133344_filtered_barcodes.tsv.gz", and "GSE133344_filtered_cell_identities.csv.gz" in the repository. The data is licensed with Open Data Commons Open Database License. The license of the data can be found here: https://catalog.data.gov/dataset/gene-expression-omnibus-geo. 

The python **anndata** package is found here: https://github.com/scverse/anndata?tab=BSD-3-Clause-1-ov-file

The **scanpy** package is found here: https://github.com/scverse/scanpy

The **requests** package is found here: https://github.com/psf/requests

In [1]:
"""
Download, read, and preprocess Norman et al. (2019) expression data.

Single-cell expression data from Norman et al. Exploring genetic interaction
manifolds constructed from rich single-cell phenotypes. Science (2019).
"""

import gzip
import os
import re

import pandas as pd
import scanpy as sc
from anndata import AnnData
from scipy.io import mmread
from scipy.sparse import coo_matrix

import requests


# Gene program lists obtained by cross-referencing the heatmap here
# https://github.com/thomasmaxwellnorman/Perturbseq_GI/blob/master/GI_optimal_umap.ipynb
# with Figure 2b in Norman 2019
G1_CYCLE = [
    "CDKN1C+CDKN1B",
    "CDKN1B+ctrl",
    "CDKN1B+CDKN1A",
    "CDKN1C+ctrl",
    "ctrl+CDKN1A",
    "CDKN1C+CDKN1A",
    "CDKN1A+ctrl",
]

ERYTHROID = [
    "BPGM+SAMD1",
    "ATL1+ctrl",
    "UBASH3B+ZBTB25",
    "PTPN12+PTPN9",
    "PTPN12+UBASH3A",
    "CBL+CNN1",
    "UBASH3B+CNN1",
    "CBL+UBASH3B",
    "UBASH3B+PTPN9",
    "PTPN1+ctrl",
    "CBL+PTPN9",
    "CNN1+UBASH3A",
    "CBL+PTPN12",
    "PTPN12+ZBTB25",
    "UBASH3B+PTPN12",
    "SAMD1+PTPN12",
    "SAMD1+UBASH3B",
    "UBASH3B+UBASH3A",
]

PIONEER_FACTORS = [
    "ZBTB10+SNAI1",
    "FOXL2+MEIS1",
    "POU3F2+CBFA2T3",
    "DUSP9+SNAI1",
    "FOXA3+FOXA1",
    "FOXA3+ctrl",
    "LYL1+IER5L",
    "FOXA1+FOXF1",
    "FOXF1+HOXB9",
    "FOXA1+HOXB9",
    "FOXA3+HOXB9",
    "FOXA3+FOXA1",
    "FOXA3+FOXL2",
    "POU3F2+FOXL2",
    "FOXF1+FOXL2",
    "FOXA1+FOXL2",
    "HOXA13+ctrl",
    "ctrl+HOXC13",
    "HOXC13+ctrl",
    "MIDN+ctrl",
    "TP73+ctrl",
]

GRANULOCYTE_APOPTOSIS = [
    "SPI1+ctrl",
    "ctrl+SPI1",
    "ctrl+CEBPB",
    "CEBPB+ctrl",
    "JUN+CEBPA",
    "CEBPB+CEBPA",
    "FOSB+CEBPE",
    "ZC3HAV1+CEBPA",
    "KLF1+CEBPA",
    "ctrl+CEBPA",
    "CEBPA+ctrl",
    "CEBPE+CEBPA",
    "CEBPE+SPI1",
    "CEBPE+ctrl",
    "ctrl+CEBPE",
    "CEBPE+RUNX1T1",
    "CEBPE+CEBPB",
    "FOSB+CEBPB",
    "ETS2+CEBPE",
]

MEGAKARYOCYTE = [
    "ctrl+ETS2",
    "MAPK1+ctrl",
    "ctrl+MAPK1",
    "ETS2+MAPK1",
    "CEBPB+MAPK1",
    "MAPK1+TGFBR2",
]

PRO_GROWTH = [
    "CEBPE+KLF1",
    "KLF1+MAP2K6",
    "AHR+KLF1",
    "ctrl+KLF1",
    "KLF1+ctrl",
    "KLF1+BAK1",
    "KLF1+TGFBR2",
]


def download_binary_file(file_url: str, output_path: str) -> None:
    """
    Download binary data file from a URL.

    Args:
    ----
        file_url: URL where the file is hosted.
        output_path: Output path for the downloaded file.

    Returns
    -------
        None.
    """
    request = requests.get(file_url)
    with open(output_path, "wb") as f:
        f.write(request.content)
    print(f"Downloaded data from {file_url} at {output_path}")




def download_norman_2019(output_path: str) -> None:
    """
    Download Norman et al. 2019 data and metadata files from the hosting URLs.

    Args:
    ----
        output_path: Output path to store the downloaded and unzipped
        directories.

    Returns
    -------
        None. File directories are downloaded to output_path.
    """

    file_urls = (
        "https://ftp.ncbi.nlm.nih.gov/geo/series/GSE133nnn/GSE133344/suppl"
        "/GSE133344_filtered_matrix.mtx.gz",
        "https://ftp.ncbi.nlm.nih.gov/geo/series/GSE133nnn/GSE133344/suppl"
        "/GSE133344_filtered_genes.tsv.gz",
        "https://ftp.ncbi.nlm.nih.gov/geo/series/GSE133nnn/GSE133344/suppl"
        "/GSE133344_filtered_barcodes.tsv.gz",
        "https://ftp.ncbi.nlm.nih.gov/geo/series/GSE133nnn/GSE133344/suppl"
        "/GSE133344_filtered_cell_identities.csv.gz",
    )

    for url in file_urls:
        output_filename = os.path.join(output_path, url.split("/")[-1])
        download_binary_file(url, output_filename)


def read_norman_2019(file_directory: str) -> coo_matrix:
    """
    Read the expression data for Norman et al. 2019 in the given directory.

    Args:
    ----
        file_directory: Directory containing Norman et al. 2019 data.

    Returns
    -------
        A sparse matrix containing single-cell gene expression count, with rows
        representing genes and columns representing cells.
    """

    with gzip.open(
        os.path.join(file_directory, "GSE133344_filtered_matrix.mtx.gz"), "rb"
    ) as f:
        matrix = mmread(f)
        
    print("read")

    return matrix


def preprocess_norman_2019(download_path: str, n_top_genes: int) -> AnnData:
    """
    Preprocess expression data from Norman et al. 2019.

    Args:
    ----
        download_path: Path containing the downloaded Norman et al. 2019 data file.
        n_top_genes: Number of most variable genes to retain.

    Returns
    -------
        An AnnData object containing single-cell expression data. The layer
        "count" contains the count data for the most variable genes. The .X
        variable contains the normalized and log-transformed data for the most variable
        genes. A copy of data with all genes is stored in .raw.
    """
    matrix = read_norman_2019(download_path)

    # List of cell barcodes. The barcodes in this list are stored in the same order
    # as cells are in the count matrix.
    cell_barcodes = pd.read_csv(
        os.path.join(download_path, "GSE133344_filtered_barcodes.tsv.gz"),
        sep="\t",
        header=None,
        names=["cell_barcode"],
    )
    

    # IDs/names of the gene features.
    gene_list = pd.read_csv(
        os.path.join(download_path, "GSE133344_filtered_genes.tsv.gz"),
        sep="\t",
        header=None,
        names=["gene_id", "gene_name"],
    )
    

    # Dataframe where each row corresponds to a cell, and each column corresponds
    # to a gene feature.
    matrix = pd.DataFrame(
        matrix.transpose().todense(),
        columns=gene_list["gene_id"],
        index=cell_barcodes["cell_barcode"],
        dtype="int32",
    )

    # Dataframe mapping cell barcodes to metadata about that cell (e.g. which CRISPR
    # guides were applied to that cell). Unfortunately, this list has a different
    # ordering from the count matrix, so we have to be careful combining the metadata
    # and count data.
    cell_identities = pd.read_csv(
        os.path.join(download_path, "GSE133344_filtered_cell_identities.csv.gz")
    ).set_index("cell_barcode")
    

    # This merge call reorders our metadata dataframe to match the ordering in the
    # count matrix. Some cells in `cell_barcodes` do not have metadata associated with
    # them, and their metadata values will be filled in as NaN.
    aligned_metadata = pd.merge(
        cell_barcodes,
        cell_identities,
        left_on="cell_barcode",
        right_index=True,
        how="left",
    ).set_index("cell_barcode")

    
    adata = AnnData(matrix)
    adata.obs = aligned_metadata

    # Filter out any cells that don't have metadata values.
    rows_without_nans = [
        index for index, row in adata.obs.iterrows() if not row.isnull().any()
    ]
    adata = adata[rows_without_nans, :]

    # Remove these as suggested by the authors. See lines referring to
    # NegCtrl1_NegCtrl0 in GI_generate_populations.ipynb in the Norman 2019 paper's
    # Github repo https://github.com/thomasmaxwellnorman/Perturbseq_GI/
    adata = adata[adata.obs["guide_identity"] != "NegCtrl1_NegCtrl0__NegCtrl1_NegCtrl0"]

    # We create a new metadata column with cleaner representations of CRISPR guide
    # identities. The original format is <Guide1>_<Guide2>__<Guide1>_<Guide2>_<number>
    adata.obs["guide_merged"] = adata.obs["guide_identity"]

    control_regex = re.compile(r"NegCtrl(.*)_NegCtrl(.*)+NegCtrl(.*)_NegCtrl(.*)")
    for i in adata.obs["guide_merged"].unique():
        if control_regex.match(i):
            # For any cells that only had control guides, we don't care about the
            # specific IDs of the guides. Here we relabel them just as "ctrl".
            adata.obs["guide_merged"].replace(i, "ctrl", inplace=True)
        else:
            # Otherwise, we reformat the guide label to be <Guide1>+<Guide2>. If Guide1
            # or Guide2 was a control, we replace it with "ctrl".
            split = i.split("__")[0]
            split = split.split("_")
            for j, string in enumerate(split):
                if "NegCtrl" in split[j]:
                    split[j] = "ctrl"
            adata.obs["guide_merged"].replace(i, f"{split[0]}+{split[1]}", inplace=True)

    guides_to_programs = {}
    guides_to_programs.update(dict.fromkeys(G1_CYCLE, "G1 cell cycle arrest"))
    guides_to_programs.update(dict.fromkeys(ERYTHROID, "Erythroid"))
    guides_to_programs.update(dict.fromkeys(PIONEER_FACTORS, "Pioneer factors"))
    guides_to_programs.update(
        dict.fromkeys(GRANULOCYTE_APOPTOSIS, "Granulocyte/apoptosis")
    )
    guides_to_programs.update(dict.fromkeys(PRO_GROWTH, "Pro-growth"))
    guides_to_programs.update(dict.fromkeys(MEGAKARYOCYTE, "Megakaryocyte"))
    guides_to_programs.update(dict.fromkeys(["ctrl"], "Ctrl"))

    # We only keep cells whose guides were either controls or are labeled with a
    # specific gene program
    adata = adata[adata.obs["guide_merged"].isin(guides_to_programs.keys())]
    adata.obs["gene_program"] = [
        guides_to_programs[x] for x in adata.obs["guide_merged"]
    ]

    adata.obs["good_coverage"] = adata.obs["good_coverage"].astype(bool)

    adata.layers["count"] = adata.X.copy()
    sc.pp.normalize_total(adata)
    sc.pp.log1p(adata)
    adata.raw = adata
    sc.pp.highly_variable_genes(
        adata, flavor="seurat_v3", n_top_genes=n_top_genes, layer="count", subset=True
    )
    adata = adata[adata.layers["count"].sum(1) != 0]  # Remove cells with all zeros.
    return adata

### Process to Get Foreground and Background  dataset

In [1]:
import time 
start = time.time()

data = preprocess_norman_2019("", 1000)

In [15]:
control = data[data.obs['guide_merged'] == 'ctrl']
control = control.to_df()
# print(control.obs)

In [19]:
CRISPR_perturbation = data[data.obs['guide_merged'] != 'ctrl']
CRISPR_perturbation = CRISPR_perturbation.to_df()
# print(CRISPR_perturbation.obs)

In [30]:
foreground = CRISPR_perturbation
background = control

foreground.to_csv("CRISPR_perturbation.csv")
background.to_csv("control_perturb_seq.csv")

n1, p = foreground.shape
n2, k = background.shape

# get the dimension for the fore and back matrix 
print("n1: ", n1)
print("n2: ", n2) 
print(p)
print(k)

n1:  24913
n2:  8907
1000
1000


### Run Contrastive Dimension Estimation Function

In [26]:
import numpy as np
import pandas as pd
import skdim
from scipy.linalg import eigh
from sklearn.preprocessing import scale


def id_estimators(df, k):
    # Maximum Likelihood algorithm
    MLE = skdim.id.MLE(K=k).fit(df).dimension_
    # Method Of Moments algorithm
    MOM = skdim.id.MOM().fit(df).dimension_
    L = {
        'MLE': MLE,
        'MOM': MOM,
    }
    return L


def est_V1_V2(X1, X2, d1, d2):
    OUT = {}
    p = X1.shape[1]
    Cx1 = np.cov(X1, rowvar=False)
    Cx2 = np.cov(X2, rowvar=False)
    # eigenvalues python package in increasing order
    val1, vectors1 = eigh(Cx1)
    idx = np.argsort(val1)
    descending_idx = idx[::-1]
    vectors1 = vectors1[:, descending_idx]
    V1 = vectors1[:, 0:d1]
    val2, vectors2 = eigh(Cx2)
    idx_ = np.argsort(val2)
    descending_idx_ = idx_[::-1]
    vectors2 = vectors2[:, descending_idx_]
    V2 = vectors2[:, 0:d2]
    OUT['V1'] = V1
    OUT['V2'] = V2
    return OUT


def sigma1_test_stat(X1, X2, d1, d2):
    OUT = est_V1_V2(X1, X2, d1, d2)
    U = OUT['V1']
    V = OUT['V2']
    M = np.matmul(U.T, V)
    _, cosines, _ = np.linalg.svd(M)
    cosines = np.minimum(1, np.maximum(-1, cosines))
    return cosines[::-1][0]     # first elt of reversed cosines list


def sing_vals(U, V):
    M = np.matmul(U.T, V)
    _, cosines, _ = np.linalg.svd(M)
    cosines = np.minimum(1, np.maximum(-1, cosines))
    return cosines


def boot_test(X1, X2, d1, d2, B):
    X1 = scale(X1, with_mean=True, with_std=False)
    X2 = scale(X2, with_mean=True, with_std=False)
    test_stat = sigma1_test_stat(X1, X2, d1, d2)
    n1 = len(X1)
    n2 = len(X2)
    boot_stats = []
    for j in range(1, B+1):
        print(j)
        idx1 = np.random.choice(range(n1), size=n1, replace=True)
        X1t = X1[idx1, :]
        combined = np.vstack((X1, X2))
        idx2 = np.random.choice(range(n1+n2), size=n2, replace=True)
        X2t = combined[idx2, :]
        boot_stats.append(sigma1_test_stat(X1t, X2t, d1, d2))
    p_value = np.mean(boot_stats < test_stat)
    return {'test_stat': test_stat, 'p_value': p_value}


def CD(X1, X2, d1, d2, epsilon=0.1, B=1000):
    p = X1.shape[1]
    OUT = est_V1_V2(X1, X2, d1, d2)
    singular_vals = sing_vals(OUT['V1'], OUT['V2'])
    singular_vals = singular_vals[::-1]
    L = {}
    L['CD'] = sum(singular_vals < 1 - epsilon) + max(d1 - d2, 0)
    test = boot_test(X1, X2, d1, d2, B)
    L['test_stat'] = test['test_stat']
    L['p_value'] = test['p_value']
    L['singular_vals'] = singular_vals
    L['d1'] = d1
    L['d2'] = d2
    return L


def CDE(fg, bg):
    L1 = id_estimators(fg, 10)
    d1 = round(L1["MOM"])
    L2 = id_estimators(bg, 10)
    d2 = round(L2["MOM"])
    return CD(fg, bg, d1, d2)


In [1]:
fore = pd.read_csv("CRISPR_perturbation.csv")
fore = fore.drop(fore.columns[0], axis=1)
back = pd.read_csv("control_perturb_seq.csv")
back = back.drop(back.columns[0], axis=1)

np.random.seed(42)
CDE(fore, back)

In [ ]:
end = time.time()
elapsed = end - start 
print(f'Time taken: {elapsed: .6f} seconds')